# ECE 445 (ML for ENGG): Mini Jupyter Exercise #2

*Eric Jiang
ewj12 158002948*

9/27/2018

In [1]:
import matplotlib as plt
import mpl_toolkits.mplot3d as plt3d
import numpy as np
%matplotlib inline

**Synthetic Data**

Problem 1

use np.random.normal(mean,variance,(rows,cols))

In [2]:
from numpy.linalg import matrix_rank
A = np.random.normal(0,1,(3,2))
print(A)
print(matrix_rank(A))


[[-1.51424476  0.82876058]
 [-0.88876059 -0.3637402 ]
 [-1.12532278  0.44479172]]
2


**Generation of Dataset #1**

Problem 1

In [15]:
''''
#This doesn't work for some reason...
X = np.empty([3,500])
V = np.empty([2,500])

for i in range(0,499):
    v = np.random.normal(0,1,(2))
    np.hstack((V,v))
    x = A@v
    np.hstack((X,x))
    
    #Alternatively could use:
    #np.c_[V,v]
    #np.c_[X,A@v]
    #use np.r_[mat1,mat2] for vstack (rows)
    #this took me forever to figure out!
'''

n = 500
V = np.zeros((2,n)) # Pre-allocate matrix
X = np.zeros((3,n))
for i in range(0,n):
    v = np.random.normal(0,1,(2))
    x = A@v
    V[:,i] = v # array V(2x500) storing vi's
    X[:,i] = x # array X(3x500) storing xi's

print('The dimensionality of array X is:', np.shape(X))

X1 = A@V
print('\nThe rank of X=A(3x2)V(2x500) is:', matrix_rank(X1))

The dimensionality of array X is: (3, 499)

The rank of X=A(3x2)V(2x500) is: 2


Since the x=Av data pts have dimensions of 3x1, with 500 cols of the 3 rowed x's then X's dimensions (3,500) confirms my expectations

Rank of 2 may possibly be due to the fact that X's multipliers A & V both can only have a rank of 2 given max 2 rows in V and max 2 cols in A

**Singular Value and Eigenvalue Decomposition of Dataset #1**

Problem 1

In [16]:
from numpy import linalg as la

B= '\033[1m'
E= '\033[0m'

LS, DS, RS = la.svd(X, full_matrices=True) 
#print(X1.shape,LS.shape,DS.shape,RS.shape)
#reconstructX = LS @ np.diag(DS) @ RS
#la.norm(reconstructX-X)

#lS = Left Singular Vectors, DS = Singular Values, RS = Right Singular Vectors
XXt = X@X.T
#XXt = np.nan_to_num(XXt)
Eigval, Eigvec = la.eig(XXt)

# X's LS = XXt's Eigvec
print(B+'\nLeft Singular Vectors of X:\n'+E, LS,'\n')
print(B+'Eigenvectors of XXt:\n'+E,Eigvec,'\n')

# XXt's Eigvals = X's DS^2
print(B+'Eigenvalues of XXt:'+E,Eigval)
print(B+'Square of Singular Values of X:'+E,np.square(DS))

# X's Norm^2 = Sum of X's DS^2
print(B+'\nFrobenius Norm or Energy of X:'+E,la.norm(X)**2)
print(B+'Sum of squares of Singular Vals of X:'+E,np.sum(DS**2))

print(B+'\nSingular Values of X:'+E,DS)
print(B+'Left Singular Vectors of X\n'+E,LS)
print(B+'Matrix A\n'+E,A)


Left Singular Vectors of X:
 [[-0.78442504  0.33423134 -0.52246221]
 [-0.28944381 -0.94229437 -0.16823675]
 [-0.5485432   0.01925433  0.83590049]] 

Eigenvectors of XXt:
 [[-0.78442504 -0.52246221  0.33423134]
 [-0.28944381 -0.16823675 -0.94229437]
 [-0.5485432   0.83590049  0.01925433]] 

Eigenvalues of XXt: [ 2.11476790e+03 -1.68935517e-13  2.42567776e+02]
Square of Singular Values of X: [2.11476790e+03 2.42567776e+02 6.54033659e-30]

Frobenius Norm or Energy of X: 2357.335671192682
Sum of squares of Singular Vals of X: 2357.3356711926817

Singular Values of X: [4.59866056e+01 1.55745875e+01 2.55740818e-15]
Left Singular Vectors of X
 [[-0.78442504  0.33423134 -0.52246221]
 [-0.28944381 -0.94229437 -0.16823675]
 [-0.5485432   0.01925433  0.83590049]]
Matrix A
 [[-1.51424476  0.82876058]
 [-0.88876059 -0.3637402 ]
 [-1.12532278  0.44479172]]


Problem 2

- None of the singular values are exactly zero because when singular value deconstruction is used there is noise in the separation. The noise causes the singular value to oscillate close to zero but not exactly zero.

- The relationship between the left singular vectors of X and the two largest singular values and the columns of A

**PCA of Dataset #1**

Problem 1


There only needs to be 2 principal components to capture all the data variation.

Since X was created by two 2-D arrays with max rank of 2 (both x,y variation), it can be assumed that the variation can only be along the 2-D space with the third dimension z being left at 0 variation. Thus there only needs to be two principal components along the x-y plane to capture all the data variation.

Problem 2

In [5]:
from numpy import array, dot, mean, std, empty, argsort
from numpy.linalg import eigh, solve
from numpy.random import randn
from matplotlib.pyplot import subplots, show

def cov(data):
    return dot(data.T, data) / data.shape[0]

#PCA with eigenvalues, also centers the mean and scales the data
def nppca(data, pc_count = None):

    data -= mean(data, 0)
    data /= std(data, 0)
    C = cov(data)
    E, V = eigh(C)
    key = argsort(E)[::-1][:pc_count]
    E, V = E[key], V[:, key]
    U = dot(data, V)
    return U, E, V

xtrans = nppca(X, 3)[0]

transmean = xtrans.mean()
print(B+'Estimated Value xk for k = 1,2,3:'+E,transmean)

Xmean = X.mean()
Xmeanvec = X.mean(0)
print(B+'\nMean of sum of X vectors:'+E,Xmean)
#print(B+'\nX vector mean entries all are ~ 0:\n'+E,Xmeanvec)

Estimated Value xk for k = 1,2,3: -9.43689570931383e-16

Mean of sum of X vectors: -1.1842378929335003e-18


Problem 3

In [6]:
#Using the pca above for matrix X
Xtrans = nppca(X, 3)[0]

#Retrieving x, y PCs for U[PCx,PCy]
ux = np.reshape(Xtrans[:,0],(3,1)) #PCx
uy = np.reshape(Xtrans[:,1],(3,1)) #PCy
uz = np.reshape(Xtrans[:,2],(3,1)) #PCz
U = np.hstack((ux,uy))

print(B+"Top two principal components U[PCx, PCy]:\n"+E,U)
#print(uz) 

Top two principal components U[PCx, PCy]:
 [[ 23.54168736   6.60748598]
 [-27.46636091   4.1291772 ]
 [  3.92467355 -10.73666318]]


Problem 4